In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
#read in main movie list data frame

df = pd.read_pickle('df_all_movies_2019_pk')
df.head()

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...


In [3]:
row_count = df.shape[0]
row_count

680

<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: At 11am on Feb 22, my access to the web site I used to scrape my data was blocked. So from here forward you may not be able to run the code. The code was working fine last week. Most of the cells have the output from last week still here. <br> <br>
Try running the first code cell to see if you are given access to the website. If you are not, do not run the remaining cells.</span>
</div>

<div class="alert alert-block alert-info">

</div>

<div class="alert alert-block alert-info">
<h3> In this notebook:</h3>
<br>
<span style="font-size:1.2em" >
    <ul>
        <li>I access a list of the 200 top all time box office revenue generating actors and create a dataframe</li>
        <li>Access the cast page for each film and create a dataframe with all cast</li>
        <li>Create a feature column labeled 'All_cast_rating' which is a meteric I developed. It's a weighted rating based on how many top 200 actors are in a film and their ranking on the the top 200 actors list</li>
        <li>Create interaction variables for wach of the top 10 actors</li>
    </ul>
</span>
</div>

<div class="alert alert-block alert-info">

</div>

In [4]:
# Access top 200 all time actors list and create dataframe

top_all_time_200_actors_df = pd.read_pickle('../top_all_time_200_actors_pk')
top_all_time_200_actors_df

,Actor
0,Samuel L. Jackson
1,"Robert Downey, Jr."
2,Scarlett Johansson
3,Tom Hanks
4,Bradley Cooper
...,...
195,Bette Midler
196,Michael Caine
197,Demi Moore
198,Karl Urban


In [5]:

"""
Use the url addresses created in 'df_all_movies_2019_pk' to access casting page for 
each film.

An examination of these pages showed that I can access cast data searching for 
id = 'cast-and-crew' and then all rows and then the text associated with tag 'a'

I place each cast member into a set. I use a set because the same indivudual
can play multiple roles in a film and I am interested only in unique individuals.

I finally place the cast set for each film into a dictionary with key equal to the film
ranking from 
"""

cast_2019_dict = {}

for i in range(row_count):
    cast_url = df.loc[i,'cast_page_url']
    
    r = requests.get(cast_url) 
    s = bs(r.text)
    cast_rows = s.find(id = 'cast-and-crew').find_all('tr')
    
    cast_set = set()
    for row in cast_rows:
        person = row.find('a').text
        cast_set.add(person)
        
    cast_2019_dict[i] = cast_set

In [6]:
cast_2019_dict[0]

{'Alan Silvestri',
 'Angela Bassett',
 'Anthony Mackie',
 'Anthony Russo',
 'Ava Russo',
 'Benedict Cumberbatch',
 'Benedict Wong',
 'Brad Semenoff',
 'Bradley Cooper',
 'Bria Kinter',
 'Brian Stultz',
 'Brie Larson',
 'Callan Mulvey',
 'Chadwick Boseman',
 'Charles Wood',
 'Chris Castaldi',
 'Chris Evans',
 'Chris Hemsworth',
 'Chris Pratt',
 'Christopher Markus',
 'Dan DeLeeuw',
 'Dan Sudick',
 'Danai Gurira',
 'Daniel Laurie',
 'Dave Bautista',
 'Dave Jordan',
 'Don Cheadle',
 'Elizabeth Olsen',
 'Emma Fuhrmann',
 'Evangeline Lilly',
 'Frank Grillo',
 'Gwyneth Paltrow',
 'Hayley Atwell',
 'Hiroyuki Sanada',
 'Jack Kirby',
 'Jacob Batalon',
 'Jacob Riehle',
 "James D'Arcy",
 'James Gunn',
 'Jeffrey Ford',
 'Jeremy Renner',
 'Jim Starlin',
 'Joe Russo',
 'John Slattery',
 'Jon Favreau',
 'Josh Brolin',
 'Josh Gold',
 'Juan Peralta',
 'Judianna Makovsky',
 'Karen Gillan',
 'Katherine Langford',
 'Kelly Port',
 'Ken Jeong',
 'Kerry Condon',
 'Kevin Feige',
 'Letitia Wright',
 'Lexi Rabe

In [7]:
len(cast_2019_dict.keys())

680

In [8]:
"""
Use the dictionary created above to calculate the 'All Cast Rating'. The rating is
based on the actors rank on the top 200 actors list. So the top actor on the list, 
Samuel Jackson gets a weighted score of 200/200 = 1. The nest actor in the list gets
a weighted score of 199/200 and so on. The score gets added for each top 200 actor in
the film

the final result for each film is placed into a list.
"""

all_movies_2019_cast_list = []


for i in range(row_count):
    movie_cast = cast_2019_dict[i]
    cast_power_rating = 0
    
    for actor in movie_cast:
        if (actor in top_all_time_200_actors_df.values):
            print(actor)
            actor_rank = (top_all_time_200_actors_df.
            index[top_all_time_200_actors_df.Actor == actor].tolist()[0]) + 1
            print(actor_rank)
            actor_relative_rank_power = (201 - actor_rank)/200
            print(actor_relative_rank_power)
            cast_power_rating += actor_relative_rank_power
            cast_power_rating = round(cast_power_rating, 1)
            
            

    all_movies_2019_cast_list.append(cast_power_rating)
    print(cast_power_rating)
    print('-----------------------\n')

all_movies_2019_cast_list

Paul Bettany
56
0.725
Chris Hemsworth
9
0.96
Robert Downey, Jr.
2
0.995
Chris Evans
7
0.97
Tom Hiddleston
53
0.74
Mark Ruffalo
17
0.92
Samuel L. Jackson
1
1.0
Natalie Portman
72
0.645
Angela Bassett
137
0.32
Benedict Cumberbatch
35
0.83
Gwyneth Paltrow
44
0.785
Josh Brolin
28
0.865
Evangeline Lilly
144
0.285
Elizabeth Olsen
48
0.765
Robert Redford
119
0.41
Chris Pratt
11
0.95
Paul Rudd
78
0.615
Sebastian Stan
50
0.755
Jon Favreau
106
0.475
Jeremy Renner
21
0.9
Benedict Wong
89
0.56
Brie Larson
113
0.44
Chadwick Boseman
25
0.88
Don Cheadle
19
0.91
Danai Gurira
64
0.685
Tom Holland
18
0.915
Karen Gillan
29
0.86
Scarlett Johansson
3
0.99
Bradley Cooper
5
0.98
Anthony Mackie
118
0.415
Pom Klementieff
79
0.61
Zoe Saldana
10
0.955
Letitia Wright
55
0.73
Dave Bautista
43
0.79
25.7
-----------------------

Keegan-Michael Key
108
0.465
BeyoncÃ© Knowles
178
0.115
Jon Favreau
106
0.475
Seth Rogen
86
0.575
1.7
-----------------------

Alan Tudyk
147
0.27
Kristen Bell
136
0.325
Josh Gad
82
0.595
1.

0
-----------------------

0
-----------------------

Melissa McCarthy
200
0.005
0.0
-----------------------

0
-----------------------

0
-----------------------

0
-----------------------

Robert Pattinson
115
0.43
0.4
-----------------------

Anne Hathaway
109
0.46
Mark Ruffalo
17
0.92
1.4
-----------------------

0
-----------------------

Brad Pitt
42
0.795
0.8
-----------------------

Tobey Maguire
117
0.42
0.4
-----------------------

0
-----------------------

0
-----------------------

Bruce Willis
23
0.89
0.9
-----------------------

0
-----------------------

Cate Blanchett
123
0.39
Kristen Wiig
162
0.195
0.6
-----------------------

0
-----------------------

Matthew McConaughey
102
0.495
Anne Hathaway
109
0.46
1.0
-----------------------

0
-----------------------

0
-----------------------

Tobey Maguire
117
0.42
0.4
-----------------------

Bill Murray
97
0.52
Adam Driver
61
0.7
1.2
-----------------------

0
-----------------------

Michael PeÃ±a
184
0.085
0.1
---------

[25.7,
 1.7,
 1.2,
 2.8,
 2.1,
 7.4,
 2.5,
 1.2,
 1.8,
 0.3,
 3.4,
 0.6,
 1.9,
 0.6,
 1.2,
 1.7,
 0.7,
 2.4,
 0.9,
 0.5,
 1.4,
 0.7,
 0.9,
 2.2,
 0,
 1.6,
 1.2,
 3.9,
 0.8,
 1.3,
 0,
 0,
 0.2,
 0.6,
 2.0,
 0,
 0,
 0,
 1.1,
 1.5,
 1.1,
 0,
 1.1,
 0.6,
 0,
 0.1,
 1.0,
 0.8,
 1.8,
 0.3,
 0,
 0,
 0,
 0.3,
 1.3,
 0,
 1.5,
 2.9,
 0,
 0,
 0.1,
 0.7,
 0.5,
 0,
 0,
 0.6,
 0,
 0,
 0,
 1.7,
 0,
 0,
 1.5,
 0.5,
 0.3,
 0.2,
 0,
 0.8,
 0.8,
 0.8,
 0,
 1.2,
 0.7,
 0,
 0.9,
 0,
 0,
 0.8,
 0.6,
 0,
 0.8,
 0,
 0,
 1.1,
 0,
 0.9,
 0.7,
 1.7,
 0,
 0,
 0,
 1.0,
 1.0,
 0.6,
 0,
 1.4,
 0,
 0.1,
 3.4,
 1.7,
 0.8,
 0,
 0,
 0.8,
 0.2,
 2.2,
 0.2,
 0.4,
 0,
 1.0,
 2.3,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0.4,
 1.4,
 0,
 0.8,
 0.4,
 0,
 0,
 0.9,
 0,
 0.6,
 0,
 1.0,
 0,
 0,
 0.4,
 1.2,
 0,
 0.1,
 0.1,
 0,
 0,
 1.7,
 0.4,
 0,
 0.9,
 0,
 0.4,
 0.5,
 0,
 0,
 0.0,
 0,
 0,
 0,
 0.8,
 0,
 0,
 0,
 0.8,
 0,
 0,
 0,
 0,
 0.6,
 0.1,
 0,
 0,
 0,
 0,
 1.0,
 0,
 0.1,
 0,
 0.6,
 0,
 0,
 1.4,
 0,
 0,
 0,
 0,
 0,
 0.8,
 0,
 0,
 0.9,
 0,
 

In [9]:
len(all_movies_2019_cast_list)

680

In [1]:
#Create a dataframe for cast data and add the all cast rating calculated above as column

cast_2019_df = pd.DataFrame(all_movies_2019_cast_list,
                                 columns = ['All_Cast_Rating'])

NameError: name 'pd' is not defined

In [12]:
cast_2019_df

,All_Cast_Rating
0,25.7
1,1.7
2,1.2
3,2.8
4,2.1
...,...
675,0.0
676,0.0
677,0.8
678,0.0


In [13]:
#Creat a list of the top 10 actors

top10_actor_list = []
for i in range(10):
    actor = top_all_time_200_actors_df.iloc[i, 0]
    top10_actor_list.append(actor)
#     print(actor)
top10_actor_list

['Samuel L. Jackson',
 'Robert Downey, Jr.',
 'Scarlett Johansson',
 'Tom Hanks',
 'Bradley Cooper',
 'Harrison Ford',
 'Chris Evans',
 'Tom Cruise',
 'Chris Hemsworth',
 'Zoe Saldana']

In [14]:
#Create columns in dataframe for each individual actor to be used as interaction 
#variables

for actor in top10_actor_list:
    cast_2019_df[actor] = 0

In [15]:
cast_2019_df

,All_Cast_Rating,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,25.7,0,0,0,0,0,0,0,0,0,0
1,1.7,0,0,0,0,0,0,0,0,0,0
2,1.2,0,0,0,0,0,0,0,0,0,0
3,2.8,0,0,0,0,0,0,0,0,0,0
4,2.1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
675,0.0,0,0,0,0,0,0,0,0,0,0
676,0.0,0,0,0,0,0,0,0,0,0,0
677,0.8,0,0,0,0,0,0,0,0,0,0
678,0.0,0,0,0,0,0,0,0,0,0,0


In [16]:
#Create interaction variable values for each actor by iterating over the cast 
#dictionary(created above) with the cast for each film.

for i in range(row_count):
    movie_cast = cast_2019_dict[i]
    
    for actor in movie_cast:
        if (actor in top10_actor_list):
            cast_2019_df.loc[i, actor] = 1
    

In [17]:
#A few checks of the the final cast dataframe

cast_2019_df.head(7)

,All_Cast_Rating,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,25.7,1,1,1,0,1,0,1,0,1,1
1,1.7,0,0,0,0,0,0,0,0,0,0
2,1.2,0,0,0,0,0,0,0,0,0,0
3,2.8,0,0,0,1,0,0,0,0,0,0
4,2.1,1,0,0,0,0,0,0,0,0,0
5,7.4,1,0,0,0,0,0,0,0,0,0
6,2.5,1,0,0,0,0,0,0,0,0,0


In [18]:
sorted_df = cast_2019_df.sort_values('All_Cast_Rating', ascending = False)

In [19]:
sorted_df

,All_Cast_Rating,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,25.7,1,1,1,0,1,0,1,0,1,1
5,7.4,1,0,0,0,0,0,0,0,0,0
27,3.9,0,0,0,0,0,0,0,0,0,0
437,3.8,0,0,0,0,0,0,0,0,0,0
10,3.4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
302,0.0,0,0,0,0,0,0,0,0,0,0
301,0.0,0,0,0,0,0,0,0,0,0,0
300,0.0,0,0,0,0,0,0,0,0,0,0
299,0.0,0,0,0,0,0,0,0,0,0,0


In [20]:
# cast_2019_df.to_pickle('cast_2019_pk')